# Running Dask on the cluster

The dask frameworks enabling users to parallelize internal systems
Not all computations fit into a big dataframe. Dask exposes lower-level APIs letting you build custom systems for in-house applications. This helps parallelize python processes and dramatically accelerate their performance

Dask Kubernetes deploys Dask workers on Kubernetes clusters using native Kubernetes APIs. It is designed to dynamically launch short-lived deployments of workers during the lifetime of a Python process.

Check out this link https://kubernetes.dask.org/en/latest/

In [2]:
!pip install dask==2.9.0 distributed==2.9.0
!pip install dask-kubernetes==0.10.0

  Using cached https://files.pythonhosted.org/packages/23/ce/8d8fe5c208e1445eafd0cc9ed9a69e761e76759c07a389d0b231c099c9d1/dask_kubernetes-0.10.0-py3-none-any.whl


When user runs dask the frameworks start one or more pods running in parallel on the cluster. Users can define the number of nodes and the minimun and maximum number of pods that the dask framework opens up
Scale to zero is achieved by setting the minimum = 0. Setting it to zero delete the pods once the job is done and free up the resources 

In [3]:
from dask_kubernetes import KubeCluster

cluster = KubeCluster.from_yaml('worker-spec.yml')
cluster.scale_up(4)  # specify number of nodes explicitly

cluster.adapt(minimum=2, maximum=5)  # or dynamically scale based on current workload

/conda/lib/python3.6/site-packages/bokeh/themes/theme.py:131: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  json = yaml.load(f)
distributed.dashboard.proxy - INFO - To route to workers diagnostics web server please install jupyter-server-proxy: pip install jupyter-server-proxy
distributed.scheduler - INFO - Clear task state
distributed.scheduler - INFO -   Scheduler at:   tcp://10.200.0.43:46055
distributed.scheduler - INFO -   dashboard at:                     :8787


to view the pods that are running

In [4]:
!kubectl  -n default-tenant get pods | grep dask

dask-iguazio-d2abc95d-8f69ss                       1/1     Running   0          10s
dask-iguazio-d2abc95d-8s4mrm                       1/1     Running   0          10s


In [5]:
# Example usage
import distributed
import dask.array as da

# Connect dask to the cluster
client = distributed.Client(cluster)

# Create an array and calculate the mean
array = da.ones((1000, 1000, 1000), chunks=(100, 100, 10))
print(array.mean().compute())  # Should print 1.0

distributed.scheduler - INFO - Receive client connection: Client-2f68ecb8-639a-11ea-9cd4-b98a59e0e5d8
distributed.core - INFO - Starting established connection
distributed.protocol.core - CRITICAL - Failed to deserialize
Traceback (most recent call last):
  File "/User/.pythonlibs/jupyter/lib/python3.6/site-packages/distributed/protocol/core.py", line 106, in loads
    header = msgpack.loads(header, use_list=False, **msgpack_opts)
  File "msgpack/_unpacker.pyx", line 195, in msgpack._cmsgpack.unpackb
ValueError: tuple is not allowed for map key
distributed.core - ERROR - tuple is not allowed for map key
Traceback (most recent call last):
  File "/User/.pythonlibs/jupyter/lib/python3.6/site-packages/distributed/core.py", line 456, in handle_stream
    msgs = await comm.read()
  File "/User/.pythonlibs/jupyter/lib/python3.6/site-packages/distributed/comm/tcp.py", line 222, in read
    frames, deserialize=self.deserialize, deserializers=deserializers
  File "/User/.pythonlibs/jupyter/lib/

CancelledError: 